# 1. Qdrant

In [1]:
! pip install -q qdrant_client==1.3.1

StatementMeta(, 6d279506-493f-4226-966f-e05e14498d80, 3, Finished, Available)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow-skinny 2.1.1 requires protobuf<5,>=3.12.0, but you have protobuf 5.26.1 which is incompatible.


In [2]:
qdrant_adress = [
    {"email":"muito1712@gmail.com",
    "qdrant_url":"https://bd26be9e-256b-4c84-85b3-2588bfdd284e.us-east-1-0.aws.cloud.qdrant.io:6333", 
    "qdrant_api_key":"qozq2_b5cqx0CI_EuDDWDUrTSEozbkQgCKplto5hlssNa064wwNKjg"},
]

from qdrant_client import QdrantClient
def get_collections_information(qdrant_url, qdrant_api_key):
    client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
    collections = client.get_collections()
    if len (collections.collections) == 0:
        print ("No collections found")
    else:
        print ("Collections found:")
        for collection in collections.collections:
            print (collection.name)


for i in range (len(qdrant_adress)):
    print ("Qdrant adress: ", qdrant_adress[i]["qdrant_url"])
    get_collections_information(qdrant_adress[i]["qdrant_url"], qdrant_adress[i]["qdrant_api_key"])
    print ("-----------------------")

StatementMeta(, 6d279506-493f-4226-966f-e05e14498d80, 4, Finished, Available)

Qdrant adress:  https://bd26be9e-256b-4c84-85b3-2588bfdd284e.us-east-1-0.aws.cloud.qdrant.io:6333
Collections found:
EP_DATABASE
CSDL_TCDK_MAIN
MARKET_REPORTS
-----------------------


In [3]:
get_collections_information(qdrant_adress[0]['qdrant_url'], qdrant_adress[0]['qdrant_api_key'])

StatementMeta(, 6d279506-493f-4226-966f-e05e14498d80, 5, Finished, Available)

Collections found:
EP_DATABASE
CSDL_TCDK_MAIN
MARKET_REPORTS


# 2. Neo4j

In [4]:
!pip install -q neo4j==5.15.0

StatementMeta(, 6d279506-493f-4226-966f-e05e14498d80, 6, Finished, Available)

In [5]:
from typing import Any, Dict, List, Optional

node_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
RETURN {labels: nodeLabels, properties: properties} AS output

"""

rel_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
RETURN {type: nodeLabels, properties: properties} AS output
"""

rel_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE type = "RELATIONSHIP" AND elementType = "node"
UNWIND other AS other_node
RETURN {start: label, type: property, end: toString(other_node)} AS output
"""


class Neo4jGraph():
    """Neo4j wrapper for graph operations.

    *Security note*: Make sure that the database connection uses credentials
        that are narrowly-scoped to only include necessary permissions.
        Failure to do so may result in data corruption or loss, since the calling
        code may attempt commands that would result in deletion, mutation
        of data if appropriately prompted or reading sensitive data if such
        data is present in the database.
        The best way to guard against such negative outcomes is to (as appropriate)
        limit the permissions granted to the credentials used with this tool.

        See https://python.langchain.com/docs/security for more information.
    """

    def __init__(
        self,
        url: Optional[str] = None,
        username: Optional[str] = None,
        password: Optional[str] = None,
        database: str = "neo4j",
    ) -> None:
        """Create a new Neo4j graph wrapper instance."""
        try:
            import neo4j
        except ImportError:
            raise ValueError(
                "Could not import neo4j python package. "
                "Please install it with `pip install neo4j`."
            )

        self._driver = neo4j.GraphDatabase.driver(url, auth=(username, password))
        self._database = database
        self.schema: str = ""
        self.structured_schema: Dict[str, Any] = {}
        # Verify connection
        try:
            self._driver.verify_connectivity()
        except neo4j.exceptions.ServiceUnavailable:
            raise ValueError(
                "Could not connect to Neo4j database. "
                "Please ensure that the url is correct"
            )
        except neo4j.exceptions.AuthError:
            raise ValueError(
                "Could not connect to Neo4j database. "
                "Please ensure that the username and password are correct"
            )
        # Set schema
        try:
            self.refresh_schema()
        except neo4j.exceptions.ClientError:
            raise ValueError(
                "Could not use APOC procedures. "
                "Please ensure the APOC plugin is installed in Neo4j and that "
                "'apoc.meta.data()' is allowed in Neo4j configuration "
            )

    @property
    def get_schema(self) -> str:
        """Returns the schema of the Graph"""
        return self.schema

    @property
    def get_structured_schema(self) -> Dict[str, Any]:
        """Returns the structured schema of the Graph"""
        return self.structured_schema

    def query(self, query: str, params: dict = {}) -> List[Dict[str, Any]]:
        """Query Neo4j database."""
        from neo4j.exceptions import CypherSyntaxError

        with self._driver.session(database=self._database) as session:
            try:
                data = session.run(query, params)
                return [r.data() for r in data]
            except CypherSyntaxError as e:
                raise ValueError(f"Generated Cypher Statement is not valid\n{e}")

    def refresh_schema(self) -> None:
        """
        Refreshes the Neo4j graph schema information.
        """
        node_properties = [el["output"] for el in self.query(node_properties_query)]
        rel_properties = [el["output"] for el in self.query(rel_properties_query)]
        relationships = [el["output"] for el in self.query(rel_query)]

        self.structured_schema = {
            "node_props": {el["labels"]: el["properties"] for el in node_properties},
            "rel_props": {el["type"]: el["properties"] for el in rel_properties},
            "relationships": relationships,
        }

        # Format node properties
        formatted_node_props = []
        for el in node_properties:
            props_str = ", ".join(
                [f"{prop['property']}: {prop['type']}" for prop in el["properties"]]
            )
            formatted_node_props.append(f"{el['labels']} {{{props_str}}}")

        # Format relationship properties
        formatted_rel_props = []
        for el in rel_properties:
            props_str = ", ".join(
                [f"{prop['property']}: {prop['type']}" for prop in el["properties"]]
            )
            formatted_rel_props.append(f"{el['type']} {{{props_str}}}")

        # Format relationships
        formatted_rels = [
            f"(:{el['start']})-[:{el['type']}]->(:{el['end']})" for el in relationships
        ]

        self.schema = "\n".join(
            [
                "Node properties are the following:",
                ",".join(formatted_node_props),
                "Relationship properties are the following:",
                ",".join(formatted_rel_props),
                "The relationships are the following:",
                ",".join(formatted_rels),
            ]
        )



StatementMeta(, 6d279506-493f-4226-966f-e05e14498d80, 7, Finished, Available)

In [6]:
# Account: tuanna.vpi@pvu.edu.vn
NEO4J_URI="neo4j+s://6bdd6ca1.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="MptgCHC9QEsJLGkPiyWsd_WkofnGAWSuxN7HpRMON50"
AURA_INSTANCEID="6bdd6ca1"
AURA_INSTANCENAME="Instance01"

TCDK_Graph = Neo4jGraph(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

TCDK_Graph.refresh_schema()
print(TCDK_Graph.schema)
data = TCDK_Graph.query("MATCH (n) RETURN n LIMIT 10")
data

StatementMeta(, 6d279506-493f-4226-966f-e05e14498d80, 8, Finished, Available)

Node properties are the following:
Issue {name: STRING, embedding: LIST, issue_id: STRING, keywords: STRING, page_numbers: STRING, year: INTEGER, link: STRING, volumn: STRING, language: STRING, abstract: STRING, conclusion: STRING, domain: STRING, doi: STRING},Author {name: STRING, embedding: LIST, phone: STRING, organization: STRING, author_id: STRING, email: STRING},Domain {name: STRING},Organization {id: STRING, name: STRING, embedding: LIST},Year {name: INTEGER},Embedding {name: STRING, embedding: LIST, issue_id: STRING, keywords: STRING, page_numbers: STRING, year: INTEGER, link: STRING, volumn: STRING, language: STRING, abstract: STRING, conclusion: STRING, domain: STRING, doi: STRING, id: STRING, phone: STRING, organization: STRING, author_id: STRING, email: STRING}
Relationship properties are the following:

The relationships are the following:
(:Issue)-[:WRITTEN_BY]->(:Author),(:Issue)-[:WRITTEN_BY]->(:Embedding),(:Issue)-[:PUBLISHED_IN]->(:Year),(:Issue)-[:HAS_DOMAIN]->(:Doma

[{'n': {'issue_id': 'PVJ.2022.09-04',
   'keywords': 'Cặn khí quyển (AR), cracking xúc tác, dầu thô Bạch Hổ, olefin nhẹ, SCT-MAT, ZSM-5.',
   'page_numbers': '24-30',
   'year': 2022,
   'link': 'https://pvj.com.vn/index.php/TCDK/article/view/826/769',
   'volumn': 'PVJ.2022.09',
   'language': 'Tiếng Việt',
   'abstract': 'Quá trình cracking trực tiếp dầu thô Bạch Hổ và cặn khí quyển (AR) trên chất xúc tác Ecat và Ecat kết hợp ZSM-5 được thực hiện ở nhiệt độ 520 - 650oC nhằm tăng năng suất tạo olefin nhẹ (olefin C2 - C4). Khi tăng nhiệt độ từ 520 - 620oC, lượng olefin nhẹ tăng 8 - 9% khối lượng đối với cả 2 loại nguyên liệu dầu thô Bạch Hổ và cặn AR. Tại nhiệt độ 650oC, có thể thu được 24 - 26% lượng olefin nhẹ. Việc bổ sung thêm ZSM-5 hỗ trợ quá trình chuyển hóa gasoline thành olefin nhẹ, giúp tăng lượng olefin C2 - C4 lên hơn 30% với 16% hiệu suất propylene. Tuy nhiên, nếu tăng nhiệt độ lên quá cao hoặc bổ sung thêm nhiều ZSM-5 sẽ sinh ra lượng lớn sản phẩm không mong muốn là khí kh

In [7]:
# Account: tuan.a.nguyen@vpi.pvn.vn
NEO4J_URI="neo4j+s://d58fde64.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="UrmWtcXfYbU1Hc-KD2Wt4Aa5nYhWdhR6XMSISn6vSHQ"
AURA_INSTANCEID="d58fde64"
AURA_INSTANCENAME="Instance01"

STVHDN_Graph = Neo4jGraph(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

STVHDN_Graph.refresh_schema()
print(STVHDN_Graph.schema)
data = STVHDN_Graph.query("MATCH (n) RETURN n LIMIT 10")
data

StatementMeta(, 6d279506-493f-4226-966f-e05e14498d80, 9, Finished, Available)

Node properties are the following:
Characteristic {emb: LIST, name: STRING},Entity {emb: LIST, name: STRING},Object {emb: LIST, name: STRING},Subject {emb: LIST, name: STRING},Date {emb: LIST, name: STRING},Person {emb: LIST, name: STRING},Field {emb: LIST, name: STRING},Activity {emb: LIST, name: STRING},Domain {emb: LIST, name: STRING},Organization {emb: LIST, name: STRING},Product {emb: LIST, name: STRING},Culture {emb: LIST, name: STRING},Geology {emb: LIST, name: STRING},nan {emb: LIST, name: STRING},Document {emb: LIST, name: STRING},Quality {emb: LIST, name: STRING},Technology {emb: LIST, name: STRING},Nation {emb: LIST, name: STRING}
Relationship properties are the following:
ĐƯỢC_THÀNH_LẬP_VÀO {emb: LIST, description: STRING},ĐƯỢC_THÀNH_LẬP_TRÊN_CƠ_SỞ {emb: LIST, description: STRING},THUỘC {emb: LIST, description: STRING},VƯỢT_QUA_KHÓ_KHĂN_ĐỂ_TRỞ_THÀNH_TỔ_CHỨC_HÀNG_ĐẦU {emb: LIST, description: STRING},NGHIÊN_CỨU_VÀ_ĐỔI_MỚI {emb: LIST, description: STRING},TÍCH_CỰC_HỘI_NHẬP {em

[{'n': {'name': 'Giá trị cốt lõi (VPI)',
   'emb': [-0.028846245259046555,
    -0.02937699481844902,
    0.027001891285181046,
    -0.00811383593827486,
    -0.027572445571422577,
    0.006574661936610937,
    -0.009659644216299057,
    0.012041383422911167,
    -0.02874009497463703,
    -0.02138921245932579,
    0.007138583343476057,
    0.04543217271566391,
    -0.012956926599144936,
    -0.024175647646188736,
    -0.009500419721007347,
    0.0033735777251422405,
    0.0010963297681882977,
    -0.028633946552872658,
    0.029589295387268066,
    0.003381870687007904,
    -0.011570342816412449,
    0.018125101923942566,
    -0.02380412258207798,
    -0.009181969799101353,
    -0.0066940803080797195,
    -0.005655801389366388,
    0.021322868764400482,
    -0.018377207219600677,
    0.006747155450284481,
    -0.01726263388991356,
    0.033941444009542465,
    -0.015139634720981121,
    -0.035878680646419525,
    0.0005547992768697441,
    0.007735676597803831,
    0.00584819819778204,


In [8]:
# Account tuanna712@gmail.com (Authenticate by Google Account)
NEO4J_URI="neo4j+s://18429959.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="6hXdC44Aa0Tm9UD3s5xGve3itUUNH9afYOcFOp2Z3ic"
AURA_INSTANCEID="18429959"
AURA_INSTANCENAME="Instance01"

EPDB_Graph = Neo4jGraph(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

EPDB_Graph.refresh_schema()
print(EPDB_Graph.schema)
data = EPDB_Graph.query("MATCH (n) RETURN n LIMIT 10")
data

StatementMeta(, 6d279506-493f-4226-966f-e05e14498d80, 10, Finished, Available)

Node properties are the following:
File {ServerRedirectedEmbedUri: STRING, LinkingUrl: STRING, LinkingUri: STRING, reportName: STRING, YearOfPublication: STRING, Length: INTEGER, ServerRedirectedEmbedUrl: STRING, ServerRelativeUrl: STRING, name: STRING, id: STRING, title: STRING, Created: STRING, embeddings: LIST},Basin {name: STRING},Block {name: STRING},Well {name: STRING},Author {name: STRING, embeddings: LIST},Contractor {name: STRING, embeddings: LIST},TypeOfFormat {name: STRING},TypesOfReport {name: STRING},StatusOfReport {name: STRING},Fields {name: STRING},Function {name: STRING},TypeOfFile {name: STRING}
Relationship properties are the following:

The relationships are the following:
(:File)-[:HAS_INFORMATION_OF]->(:Basin),(:File)-[:HAS_INFORMATION_OF]->(:Block),(:File)-[:HAS_INFORMATION_OF]->(:Well),(:File)-[:IS_WRITTEN_BY]->(:Author),(:File)-[:IS_IMPLEMENTED_BY]->(:Contractor),(:File)-[:HAS_REPORT_TYPE]->(:TypesOfReport),(:File)-[:HAS_FINAL_STATUS]->(:StatusOfReport),(:File)

[{'n': {'embeddings': [-0.01731603778898716,
    -0.015469349920749664,
    0.020019158720970154,
    -0.036799926310777664,
    -0.019216250628232956,
    -0.0016953055746853352,
    -0.02399355173110962,
    -0.004984716884791851,
    -0.02447529509663582,
    -0.009561289101839066,
    0.008042456582188606,
    0.008296710439026356,
    0.004586608614772558,
    -0.007955474779009819,
    -0.009514452889561653,
    0.01191648468375206,
    0.02207995392382145,
    -0.014010734856128693,
    0.00321497512049973,
    -0.017115309834480286,
    -0.022494791075587273,
    0.02348504215478897,
    -0.007995620369911194,
    0.013027173466980457,
    0.000428635481512174,
    0.014653060585260391,
    0.03088517300784588,
    -0.029306121170520782,
    0.013876916840672493,
    -0.014465715736150742,
    0.015817277133464813,
    0.0011257431469857693,
    -0.027084743604063988,
    -0.009173217229545116,
    -0.011742521077394485,
    -0.035997018218040466,
    -0.00743358489125967,
    

In [10]:
# Account: tuan.a.nguyen@vpi.pvn.vn
NEO4J_URI="neo4j+s://f6724121.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="hsPy2R3_HNAtQu_lJHWpdiAgLdi2OpeBv1D6d8uJGZk"

VPINS_Graph = Neo4jGraph(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

VPINS_Graph.refresh_schema()
print(VPINS_Graph.schema)
data = VPINS_Graph.query("MATCH (n) RETURN n LIMIT 10")
data

StatementMeta(, 6d279506-493f-4226-966f-e05e14498d80, 12, Finished, Available)

Node properties are the following:
Block {id: STRING, name: STRING, geojson: STRING, area: STRING, description: STRING, status: STRING},Well {id: STRING, name: STRING, geojson: STRING, field_id: STRING, full_name: STRING, operator_id: STRING, well_alias: STRING, well_type: STRING},Prospect {id: STRING, name: STRING, description: STRING, operator_id: STRING, class: STRING},WindOffshoreProject {id: STRING, name: STRING, geojson: STRING, status: STRING, capacity: STRING},Investor {id: STRING, name: STRING},Technology {id: STRING, name: STRING, type: STRING}
Relationship properties are the following:

The relationships are the following:
(:Block)-[:CÓ_GIẾNG_KHOAN]->(:Well),(:Block)-[:CÓ_CẤU_TẠO]->(:Prospect),(:Well)-[:KHOAN_VÀO_CẤU_TẠO]->(:Prospect),(:Well)-[:THUỘC_LÔ]->(:Block),(:Prospect)-[:CÓ_GIẾNG_KHOAN]->(:Well),(:Prospect)-[:THUỘC_LÔ]->(:Block),(:WindOffshoreProject)-[:THUỘC_VỀ]->(:Investor),(:WindOffshoreProject)-[:THUỘC_NHÓM]->(:Technology),(:Investor)-[:ĐẦU_TƯ]->(:WindOffshoreProj

[{'n': {'geojson': '{"type":"Feature","geometry":{"type": "Polygon", "coordinates": [[[107.166667, 10.183333], [107.714528, 10.413778], [107.714528, 10.183333], [107.166667, 10.183333]]]}}',
   'area': '768.20',
   'name': '15-1/15',
   'description': 'Lô 15-1/15 nằm ở ngoài khơi Việt Nam  thuộc bể Cửu Long  ranh giới tiếp giáp với các lô 15-1/05  15-2/17  có diện tích khoảng 768 2 km2.',
   'id': 'CL11',
   'status': 'Open'}},
 {'n': {'geojson': '{"type":"Feature","geometry":{"type": "Polygon", "coordinates": [[[108.45, 10.139583], [108.45, 10], [108.383333, 10], [108.383333, 9.985742], [108.282333, 9.9], [108.183333, 9.9], [108.183333, 10], [108.45, 10.139583]]]}}',
   'area': '417.50',
   'name': '15-2',
   'description': 'Lô 15-2 nằm ở ngoài khơi Việt Nam  thuộc bể Cửu Long  ranh giới tiếp giáp với các lô 01&02  02/10  09-2/09  09-2  09-2/10  15-2/17  có diện tích khoảng 417 5 km2. Hiện tại lô 15-2 được điều hành bởi JVPC.',
   'id': 'CL12',
   'status': 'Close'}},
 {'n': {'geojson